# Introduction to this "tutorial"

This notebook is an interactive example for a "simple" **staircase sweep measurments experiment**, it can be loosely mapped to Programming Example in pp-198 of B1500A mannual.

Experimental configurations are:

**Active SMUs: Terminals of MOSFET:**
* SMU1 - **channel 1** - HPSMU: **Drain**  
* SMU2 - **channel 3** - HPSMU: **Gate**
* GND - channel xx - GNDU: **Source** 
    * (GNDU is not explicitly declared either in Pymeausure of Programming Manual, here we treat it as a **"dont care"**)

**Staircase Sweep Setups:**
* **Drain Voltage** "sweep" from **1v to 3v, with a step of 1v (nop1 = 3)**
    * For each step of Drain Voltage sweep: **Gate Voltage sweep** from **0v to 3v, with a step of 0.33v (nop2 = 10)**
    
**Measurements:**
We hope to measure: ```Vg, Ig, Vd, Id``` and we can measure all of these values. ```Ig, Id``` are meaningless for dummy runs without the *device under test*. **```Vg or Vd```** measurments are **consistent with expected values**. 

* However, we haven't figured out how to measure Id and Vd simutaneously. 
    * A possible workaround is to measure ```Id```, as we already know the values of ```Vd```. 

* In this interactivate example, **A spreadsheet of ```Vg, Ig, Id``` will be generated** when the experiment ends. 


**TODOs:**
* Figure out how to extract the measurements of ```Vd and Id``` simutaneously.

## 1. Setup environment

In [1]:
import pymeasure
from pymeasure.adapters import VXI11Adapter
from pymeasure.instruments import Instrument
# from pymeasure.instruments.agilent import AgilentB1500
from agilent1505 import b1505a
adapter = VXI11Adapter("TCPIP::10.8.128.194::gpib0,17::INSTR")

## 2. Connect Instrument via adapter

In [2]:
instr = b1505a(adapter, timeout='None')
instr.reset()

**Example from programmnig manual pp-199: L1-31**
<img src="jupyter_resources/sweepCodeExample1_1to31.png" alt="Drawing" style="width: 800px;"/>

Query SMUs being used

In [3]:
res = instr.query_modules()
print(f'SMUs used: {res}')

SMUs used: {1: 'HPSMU', 3: 'HPSMU', 5: 'MFCMU', 6: 'HCSMU', 8: 'HVSMU'}


**Example from programmnig manual pp-199: L32-35**

Sets the data output format, time stamp data output mode, SMU filter, and averaging.

<img src="jupyter_resources/sweepCodeExample1_32to35.png" alt="Drawing" style="width: 900px;"/>

**L32:** ```FMT``` mapped by ```instr.data_format()``` method.

* ```data_format(output_format, mode=0)``` Specifies data output format. Check Documentation for parameters. Should be called once per session to set the data format for interpreting the measurement values read from the instrument.
    * Currently implemented are format: 1, 11, and 21
* **Parameters**:
    * ```output_format (str)``` – Output format string, e.g. FMT21, seems that int may also work
    * ```mode (int, optional)``` – Data output mode, defaults to 0 (only measurement data is returned)


**L33:** ```TSC``` mapped by ```instr.time_stamp```


**L34:** ```FL``` mapped by ```smu.filter```


**L35:** ```AV``` mapped by ```adc_averaging```: Check pp-365 for details of ```AV``` command
* ```adc_averaging(number, mode='Auto')``` Set number of averaging samples of the HSADC. (AV)
* **Parameters**:
    * ```number (int)``` - Number of averages
    * ```mode (AutoManual, optional)``` – Mode ('Auto','Manual'), defaults to ‘Auto’

In [4]:
instr.time_stamp = False # Disable Timestamp
instr.set_filter() 
instr.adc_averaging(10) # Set number of averaging samples of the HSADC

## 3. Initialize SMUs
Let's stick with the terminal mapping between SMU and MOSFET defined in RER legacy codes: 
* HPSMU1<==>Channel(1) maps to **Drain**
* HPSMU2<==>Channe3(3) maps to **Gate**
* Source is set to GND (0) maps to **Source**

enable all smus, and query smunames ("smui") with corresponding channels

In [5]:
instr.initialize_all_smus()

In [6]:
for i, smu in enumerate(instr.smu_references, start=1):
    smu.enable() # enable SMUs
    print(f'smu{i} stands for channel {smu.channel}')
instr.data_format(21, mode=1)

smu1 stands for channel 1
smu2 stands for channel 3
smu3 stands for channel 6
smu4 stands for channel 8


In [7]:
print(instr.smu_references)
print(instr.smu1)
instr.smu3.disable() # Disable SMUs we dont use
instr.smu4.disable() # Disable SMUs we dont use

dict_values([<agilent1505.rerSMU object at 0x7fbb62971cd0>, <agilent1505.rerSMU object at 0x7fbb66a86b50>, <agilent1505.rerSMU object at 0x7fbb66aa43d0>, <agilent1505.rerSMU object at 0x7fbb66aa4f90>])


In [8]:
# set the alias of smus
drain = instr.smu1
gate = instr.smu2 
source = instr.GNDU  

**Example from programmnig manual pp-199: L35-37**

Applies voltage to device

<img src="jupyter_resources/sweepCodeExample1_35to37.png" alt="Drawing" style="width: 900px;"/>
In this case, we dont have the substrate, just try to setup the drain, though it might not be necessary

**L35**: ```DV``` mapped by ```force(source_type, source_range, output, comp='', comp_polarity='', comp_range='')```

In [ ]:
source.force('VOLTAGE', 0, vd, comp=0.01) # Force drain to 0

## 4. Setup measurements

**Example from programmnig manual pp-200: L38-40**

Sets the measurement mode, channel measurement mode, and measurment range

<img src="jupyter_resources/sweepCodeExample1_38to40.png" alt="Drawing" style="width: 900px;"/>

**L38:** ```MM``` mapped by ```meas_mode(mode, *args)```

* Set Measurement mode of channels. Measurements will be taken in the same order as the SMU references are passed. (MM)
* **Parameters**: 
    * mode(MeasMode)-
        * SPOT
        * STAIRCASE_SWEEP
        * SAMPLING
    * args(SMU) - SMU references


**L39:** ```CMM``` mapped by ```meas_op_mode```
* **Usage:** ```smu.meas_op_mode='MODE'```    
* **MODE list**: 
    * COMPLIANCE_SIDE= 0
    * CURRENT= 1
    * VOLTAGE= 2
    * FORCE_SIDE= 3
    * COMPLIANCE_AND_FORCE_SIDE= 4
    
**L40:** ```RI``` mapped by ```meas_range_current```
* **Usage:**```smu.meas_range_current='RANGE```
    * ```Range``` was defined in ```rerSMUCurrentRanging```

In [9]:
instr.meas_mode('STAIRCASE_SWEEP', *instr.smu_references, channels=[3, 1]) # MM 2,
# gate.meas_op_mode=True
# instr.meas_mode('SAMPLING', *instr.smu_references, channels=[3])
# drain.meas_op_mode = 'CURRENT'
# drain.meas_range_current = 0

arugment:  (<agilent1505.rerSMU object at 0x7fbb62971cd0>, <agilent1505.rerSMU object at 0x7fbb66a86b50>, <agilent1505.rerSMU object at 0x7fbb66aa43d0>, <agilent1505.rerSMU object at 0x7fbb66aa4f90>)
channel 1
excecuted: MM 2, 1
channel 3
excecuted: MM 2, 1, 3
channel 6
channel 8


In [10]:
# drain.meas_op_mode = 'CURRENT'
# drain.meas_range_current = 0
# gate.meas_op_mode = 'VOLTAGE'
# gate.meas_range_voltage = 0

**Example from programmnig manual pp-200: L41-45**

Sets the timing parameters and sweep mode of the staircase sweep source. And checks
if an error occurred. If an error is detected, forces 0 V and goes to Check_err.

<img src="jupyter_resources/sweepCodeExample1_41to45.png" alt="Drawing" style="width: 900px;"/>

**L41**: ```WT``` mapped by ```sweep_timing()```

* ```sweep_timing(hold, delay, step_delay=0, step_trigger_delay=0, measurement_trigger_delay=0)``` Sets Hold Time, Delay Time and Step Delay Time for staircase or multi channel sweep measurement. (WT) If not set, all parameters are 0.

* **Parameters**: 
    * ```hold (float)``` - Hold Time
    * ```delay (float)``` - Delay Time
    * ```step_delay (float, optional)``` – Step delay time, defaults to 0
    * ```step_trigger_delay (float, optional)``` – Trigger delay time, defaults to 0
    * ```measurement_trigger_delay (float, optional)``` – Measurement trigger delay time, defaults to 0
    
**L42**: ```WM``` mapped by ```sweep_auto_abort()```
* **Usage:** ```sweep_auto_abort(abort, post='START')``` 
    * Enable by pass ```True``` to ```abort``` (True stands for 2)
    * post values: ```START=1```, ```STOP=2``` 

**L43**: ```ERR``` should mapped by ```b1500.check_errors()```

In [11]:
instr.sweep_timing(0, 0, step_delay=0) # All default values
instr.sweep_auto_abort(True, post='START') # WM 2,1
instr.check_errors()

**Example from programmnig manual pp-200: L46-66**

Sets the sweep source, applies voltage to device, resets time stamp, and performs the
staircase sweep measurement. And stores the returned data into the mret string
variable. Finally, stores the measured data into the data array.

<img src="jupyter_resources/sweepCodeExample1_46to66.png" alt="Drawing" style="width: 900px;"/>

**L46**: ```WV``` mapped by ```staircase_sweep_source(source_type, mode, source_range, start, stop, steps, comp, Pcomp='')```, with ```mode='Voltage'```

Specifies Staircase Sweep Source (Current or Voltage) and its parameters. (WV or WI)

* **Parameters:** 
    * ```source_type (str)``` – Source type ('Voltage','Current')
    * ```mode (SweepMode)``` – Sweep mode
        * LINEAR_SINGLE=1
        * LOG_SINGLE=2
        * LINEAR_DOUBLE=3
        * LOG_DOUBLE=4
    * ```source_range (int)``` – Source range index
    * ```start (float)``` – Sweep start value
    * ```stop (float)``` – Sweep stop value
    * ```Steps (int)``` – Number of sweep steps
    * ```comp (float)``` – Compliance value
    * ```Pcomp (float, optional)``` – Power compliance, defaults to not set
    
**L48**: ```DV``` mapped by ```force(source_type, source_range, output, comp='', comp_polarity='', comp_range='')```, with ```mode='Voltage'```

Applies DC Current or Voltage from SMU immediately. (DI, DV)

* **Parameters:** 
    * ```source_type (str)``` – Source type ('Voltage','Current')
    * ```source_range (int)``` – Source range index
    * ```output``` – Source output value in A or V
    * ```comp (float, optional)``` – Compliance value, defaults to previous setting
    * ```comp_polarity (CompliancePolarity)``` – Compliance polairty, defaults to auto
    * ```comp_range (int or str, optional)``` – Compliance ranging type, defaults to auto
    
    
**L49**: ```TSR``` mapped by ```clear_timer()```

**L50**: ```XE``` mapped by ```send_trigger()```

**L51**: ```OPC``` mapped by ```check_idle()```

**L52**: ```ERR``` mapped by ```check_errors()```

In [12]:
nop = 10
nopd = 3

In [13]:

# drain.staircase_sweep_source('VOLTAGE','LINEAR_SINGLE','Auto Ranging',0,3,nopd,0.001)
drain.meas_op_mode = 'VOLTAGE'
drain.meas_range_voltage = 0
# drain.meas_op_mode = 'CURRENT'
# drain.meas_range_current = 0
# drain.force('VOLTAGE', 0, 3, comp=0.01) # Set gate to 3v, autoranging, with a comp of 0.01

In [14]:
# #Start Measurement
# instr.check_errors()
# instr.clear_buffer()
# instr.clear_timer()
# instr.send_trigger()

## 5. Retreive Data
From this point, we fully rely on the data reader by Pymeasure

In [15]:
meas = []
vd = 1
for i in range(nopd):
#     instr.send_trigger()
#     data = instr.read_channels()
    gate.staircase_sweep_source('VOLTAGE','LINEAR_SINGLE','Auto Ranging',0,3,nop,0.001)
#     drain.force('VOLTAGE', 0, vd, comp=0.01)
    instr.check_errors()
    instr.clear_buffer()
    instr.clear_timer()
    instr.send_trigger()
    data = instr.read_channels()
    meas.append(data)
    vd = vd + 1

In [16]:
print(meas)

[(('000', 'SMU1', 'Voltage Measurement (V)', -0.0002), ('000', 'SMU2', 'Current Measurement (A)', -5e-13), ('  W', 'SMU2', 'Voltage Output (V)', 0.0), ('000', 'SMU1', 'Voltage Measurement (V)', -0.0002), ('000', 'SMU2', 'Current Measurement (A)', 6e-13), ('  W', 'SMU2', 'Voltage Output (V)', 0.333), ('000', 'SMU1', 'Voltage Measurement (V)', -0.0001), ('000', 'SMU2', 'Current Measurement (A)', -4e-13), ('  W', 'SMU2', 'Voltage Output (V)', 0.667), ('000', 'SMU1', 'Voltage Measurement (V)', -0.0002), ('000', 'SMU2', 'Current Measurement (A)', 4e-13), ('  W', 'SMU2', 'Voltage Output (V)', 1.0), ('000', 'SMU1', 'Voltage Measurement (V)', -0.0003), ('000', 'SMU2', 'Current Measurement (A)', -1.5e-13), ('  W', 'SMU2', 'Voltage Output (V)', 1.333), ('000', 'SMU1', 'Voltage Measurement (V)', -0.0002), ('000', 'SMU2', 'Current Measurement (A)', 5e-14), ('  W', 'SMU2', 'Voltage Output (V)', 1.667), ('000', 'SMU1', 'Voltage Measurement (V)', -0.0003), ('000', 'SMU2', 'Current Measurement (A)', 1

In [17]:
v_gate = []
v_drain = []
i_drain = []
i_gate = []

smu_dict = {
    'drain': 'SMU1',
    'gate': 'SMU2'
}

In [18]:
for mea in meas:
    for ele in mea:
    #     print(ele)
        if ele[1] == smu_dict['drain'] and 'Voltage' in ele[2]:
            v_drain.append(ele[3])
        if ele[1] == smu_dict['drain'] and 'Current' in ele[2]:
            i_drain.append(ele[3])
        if ele[1] == smu_dict['gate'] and 'Current' in ele[2]:
            i_gate.append(ele[3])
        if ele[1] == smu_dict['gate'] and 'Voltage' in ele[2]:
            v_gate.append(ele[3])

print(v_gate)
print(i_gate)
print(v_drain)
print(i_drain)


[0.0, 0.333, 0.667, 1.0, 1.333, 1.667, 2.0, 2.333, 2.667, 3.0, 0.0, 0.333, 0.667, 1.0, 1.333, 1.667, 2.0, 2.333, 2.667, 3.0, 0.0, 0.333, 0.667, 1.0, 1.333, 1.667, 2.0, 2.333, 2.667, 3.0]
[-5e-13, 6e-13, -4e-13, 4e-13, -1.5e-13, 5e-14, 1.5e-13, 3e-13, -1e-13, 2.5e-13, -3e-13, 2.5e-13, 1.5e-13, 1e-13, -5e-14, 2.5e-13, -2e-13, 2.5e-13, -1.5e-13, 5e-14, -1e-13, 2.5e-13, -5.5e-13, 6.5e-13, -3.5e-13, 2e-13, -5e-14, 0.0, -1.5e-13, 5e-14]
[-0.0002, -0.0002, -0.0001, -0.0002, -0.0003, -0.0002, -0.0003, -0.0002, -0.0002, -0.0002, -0.0002, -0.0002, -0.0002, -0.0002, 0.0, -0.0002, -0.0002, -0.0001, -0.0002, -0.0002, -0.0001, -0.0002, -0.0001, -0.0001, -0.0002, -0.0002, -0.0002, -0.0001, -0.0001, -0.0002]
[]


In [ ]:
import pandas as pd
import numpy as np
aggregated_data = np.array([v_gate, v_drain, i_drain, i_gate]).T.tolist()
res = pd.DataFrame(aggregated_data, columns=['vg', 'vd' ,'id', 'ig'])

In [ ]:
res